In [6]:
from perturbation_utils import *
from datasets import Dataset, load_from_disk, DatasetDict
import json


In [7]:
MODEL = 'bert'
DATA = 'hatexplain_neutral'

In [8]:
from_path_dataset = f"./Data/Clean/{DATA}"


dataset = load_from_disk(from_path_dataset)['test'] #[:250] #['test']
# dataset = Dataset.from_dict(dataset)

# annos_file = f"./Hotflip/{MODEL}-{DATA}.json"

# with open(annos_file) as json_file:
#     annos = json.load(json_file)

load_twitter_ids(dataset)
load_gradients(MODEL, DATA)


In [9]:
new_dataset = dataset.map(insert_gradient_noise)
new_dataset

ValueError: operands could not be broadcast together with shapes (18,) () (19,) 

## Debugging

In [10]:
%debug

> /home/fvd442/project/noise-paper/perturbation_utils.py(656)create_gradient_masks()
    654         mask = [True if ele in anno[-round(len(anno)*prop):] else False for ele in np.arange(len(anno))]
    655         #mask = [True if ele in anno[:round(len(anno)*prop)] else False for ele in np.arange(len(anno))]
--> 656         noisy[f'gradient-{MODEL}_token_{prop*100:02.0f}'] = ' '.join(np.where(mask, '{TOKEN}', example))
    657         noisy[f'gradient-{MODEL}_charswap_{prop*100:02.0f}'] = ' '.join(np.where(mask, random_charswap(example), example))   #' '.join(np.where(mask,  , example))
    658         noisy[f'gradient-{MODEL}_synonym_{prop*100:02.0f}'] = ' '.join(np.where(mask, wordswap(example), example))   #' '.join(np.where(mask, wordswap(example) , example))

[False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False]
['<user>', '😂', '😂', 'tell', 'me', 'about', 'it', 'they', 'have', 'been', 'co', 'opted', 'by', 'im

In [12]:
def create_gradient_masks(example, anno, MODEL):
    noisy = {}

    for prop in [0.25,0.5,0.7,0.8,0.9,0.95]:
        mask = [True if ele in anno[-round(len(anno)*prop):] else False for ele in np.arange(len(anno))]
        #mask = [True if ele in anno[:round(len(anno)*prop)] else False for ele in np.arange(len(anno))]
        noisy[f'gradient-{MODEL}_token_{prop*100:2.0f}'] = ' '.join(np.where(mask, '{TOKEN}', example))  
        noisy[f'gradient-{MODEL}_charswap_{prop*100:2.0f}'] = ' '.join(np.where(mask, charswap(example), example))   #' '.join(np.where(mask,  , example))     
        noisy[f'gradient-{MODEL}_synonym_{prop*100:2.0f}'] = ' '.join(np.where(mask, wordswap(example), example))   #' '.join(np.where(mask, wordswap(example) , example))    
        noisy[f'gradient-{MODEL}_PATTERN_{prop*100:2.0f}'] = ' '.join(mask)
    
    return noisy

In [13]:
def insert_gradient_noise(example):
    global pos_tags
    global annos
    global MODEL
    nltk_tokens = nltk.word_tokenize(example['text'])
    pos_tags = [val[1] for val in nltk.pos_tag(nltk_tokens)]
    
    tokens = example['text'].split()
    ind = example['index']
    anno = annos[str(ind)]
    
    noise = create_gradient_masks(tokens, anno, MODEL)
    return example | noise

In [14]:
def wordswap(iterable):
    global pos_tags
    global NO_MATCH
    global no_match_counter
    global word_counter
    out = []
    for i,x in enumerate(iterable):
        word_counter += 1
        new = find_replacement(x, pos_tags[i])
        if new == x:
            NO_MATCH.add((x, pos_tags[i]))
            no_match_counter += 1
        out.append(new)
                
    return out

In [16]:
new_dataset

Dataset({
    features: ['label', 'text', 'annotations', 'index', 'gradient-bert_token_25', 'gradient-bert_charswap_25', 'gradient-bert_synonym_25', 'gradient-bert_token_50', 'gradient-bert_charswap_50', 'gradient-bert_synonym_50', 'gradient-bert_token_70', 'gradient-bert_charswap_70', 'gradient-bert_synonym_70', 'gradient-bert_token_80', 'gradient-bert_charswap_80', 'gradient-bert_synonym_80', 'gradient-bert_token_90', 'gradient-bert_charswap_90', 'gradient-bert_synonym_90', 'gradient-bert_token_95', 'gradient-bert_charswap_95', 'gradient-bert_synonym_95'],
    num_rows: 63
})

In [17]:
new_dataset[19]

{'label': 0.0,
 'text': 'there are simply too many ideas floating around -- part farce part sliding doors part pop video -- and yet failing to exploit them',
 'annotations': '0.0 0.0 0.0 -0.3333 -0.3333 0.0 0.0 0.0 0.0 -0.3333 -0.3333 -0.3333 -0.3333 -0.3333 -0.3333 -0.3333 -0.3333 0.0 0.0 0.0 -1.0 -0.3333 -0.3333 -0.3333',
 'index': 121,
 'gradient-bert_token_25': '{TOKEN} are simply too many {TOKEN} {TOKEN} {TOKEN} {TOKEN} {TOKEN} farce part sliding doors part pop video -- and yet failing to exploit them',
 'gradient-bert_charswap_25': 'therBe are simply too many Mideas flYoating aroundH H-- partT farce part sliding doors part pop video -- and yet failing to exploit them',
 'gradient-bert_synonym_25': 'in-that-respect are simply too many idea float some , role farce part sliding doors part pop video -- and yet failing to exploit them',
 'gradient-bert_token_50': '{TOKEN} {TOKEN} simply too many {TOKEN} {TOKEN} {TOKEN} {TOKEN} {TOKEN} {TOKEN} {TOKEN} {TOKEN} {TOKEN} part pop video -- 